get cell extension?


In [ ]:

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: ' + (cell.filename || notebook) + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

A couple of rules:
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable in isolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```

 - eliminate circular dependencies the same way unintended recursion is avoided, create a condition:
 
ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
var funcA = require('ModuleA.js');
```

Becomes:

ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
if(typeof funcA === 'undefined') {
    var funcA = require('ModuleA.js');
}
```

 - parameters are listed in most specific left to least specific right, i.e. function(filter, context) would mean filter is only used for this function, whereas context may be passed in to this function as well as other functions.  "filter" is on the left because it is specifically used just for this function, context is on the right because it might contain a path on the filesystem, or some options.
 - Entry cells, i.e. cells that are intended to be called by a service, coordinate interactions between multiple services - should not contain catch blocks so that the task scheduler will fail and log the last error.




get imports recursively?

TODO: use this in file-system graph

TODO: AST compatible map of import functions across all languages
Python `__import__`, Java: `import`, C#: `using`, C++: `#import`.  Basic functionality of JetBrains intellisense.



In [ ]:
var importer = require('../Core');
var importerCode = importer.interpret('import notebook.ipynb');
var getArrayAST = importer.import('get ast path array');
var getExports = importer.import('get exports from source');

var GET_IMPORTS = `//CallExpression[//Identifier[@name=='import' || @name=='importNotebook']]//Literal`;
var GET_IMPORTER = `//CallExpression//Identifier[
    @name=='interpret' || @name=='runAllPromises' || @name=='runInNewContext' ||
    @name=='getCells' || @name=='streamJson' || @name=='regexToArray']`

function matchImports(code) {
    var imports = getArrayAST(GET_IMPORTS, code)
        .map(s => s.value);
    var coreMatches = getArrayAST(GET_IMPORTER, code)
        .map(s => s.name)
        .map(i => importerCode.filter(core => core.code.match(new RegExp('exports.*?' + i, 'ig')))[0].id);
    return imports.concat(coreMatches);
}

function getImportsRecursively(interpretedImports, results, searches) {
    var imports = {};
    searches.forEach((search, i) => {
        var result = typeof results[i][0] === 'undefined'
            ? [results[i]]
            : results[i];
        if(typeof interpretedImports[search] !== 'undefined') return;
        interpretedImports[search] = results[i];
        result.forEach(cell => {
            cell.exports = getExports(cell.code) || [];
            interpretedImports[cell.id] = cell;
            imports[cell.id] = matchImports(cell.code);
            imports[cell.id].forEach(arr => {
                // normalize input incase importing an array like .import(['search'])
                if(typeof arr === 'string') arr = [arr];
                var result = getImportsRecursively(interpretedImports, importer.interpret(arr), arr);
                Object.assign(imports, result);
            });
        })
    });
    
    return imports;
}

module.exports = getImportsRecursively;



rename cell to a nice name?

In [ ]:
var importer = require('../Core');
var getExtension = importer.import('cell extension')

// get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    cell.questions.sort((a, b) => a.length - b.length);
    return cell.questions[0].replace('?', '').replace(/[^a-z0-9]+|\btest\b/igm, ' ').trim().replace(/\s+/igm, '-') + getExtension(cell);
}

module.exports = niceName;


include require like import?

In [ ]:
module.exports = (r) => {
    return typeof r === 'function' ?
        r
        : (r && typeof r[Object.keys(r)[0]] === 'function'
            ? r[Object.keys(r)[0]]
            : r);
}

add previously imported code?

In [ ]:
var importer = require('../Core');
var niceName = importer.import('rename cell nice name');
var getArrayAST = importer.import('get ast path array');
var include = importer.import('include require like import');

var importTemplate = (ast, cell) => (ast.push(getArrayAST('/*', `
var ${cell.exports[0]} = (${include.toString()})(require('./${niceName(cell)}'));`)), ast);

var filterImports = (ast, cell) => cell.exports[0]
   && cell.questions.length > 0
   && 0 < getArrayAST(`//CallExpression//Identifier[@name == '${cell.exports[0]}']`, ast).length
   && 0 === getArrayAST(`//*[@type == 'VariableDeclarator' || @type == 'FunctionDeclaration']
        /Identifier[@name == '${cell.exports[0]}']`, ast).length

// TODO: use this with a polyfill across ALL notebooks on require('../Core')
// include all functional references from cells above this current cell
function addImports(ast, imports) {
    return Object.values(imports).flat()
        .filter(cell => filterImports(exportedCode, cell))
        .reduce((code, cell) => importTemplate(code, cell), ast);
}

module.exports = addImports;


replace notebook imports?

automatic jupyter dependency injector?


In [ ]:
var escodegen = require('escodegen');
var importer = require('../Core');
var addImports = importer.import('add previously imported code');
var niceName = importer.import('rename cell nice name');
var getArrayAST = importer.import('get ast path array');
var include = importer.import('include require like import');

var GET_CORE = `/VariableDeclaration[//Literal[index(@value, 'Core') > 0]]`;
var GET_CORENAME = `/*/VariableDeclarator/Identifier`;

function replaceCore(ast) {
    var impParent = getArrayAST(`//*[${GET_CORE}]`, ast)[0];
    if(impParent) {
        var impDeclare = getArrayAST(GET_CORE, impParent)[0];
        var impName = getArrayAST(GET_CORENAME, impDeclare)[0].name;
        impParent.splice(impParent.indexOf(impDeclare), 1);
        var GET_EXP = `/*[(@type == 'StaticMemberExpression' || @type == 'MemberExpression')
                && //Identifier[@name == '${impName}']]/CallExpression`;
        var impUsage = getArrayAST(`//CallExpression[${GET_EXP}]`, ast);
        impUsage.forEach(parent => {
            var impExp = getArrayAST(GET_EXP, parent);
            impExp.forEach(child => {
                var index = parent.indexOf(child);
                parent.callee = child.property;
            })
        });
    }
    return ast;
}

var multipleImportTemplate = (imports) => `[${imports}].reduce((acc, req, i) => {
    var r = (${include.toString()})(req);
    if(typeof r === 'function') {
        acc[r.name] = r;
    }
    acc[i] = r;
    return acc;
}, {})`

function replaceImport(ast, i, allImports) {
    let result = typeof i === 'string' ? allImports[i] : i.map(i => allImports[i]);
    if (typeof result === 'undefined' || result.length === 0) {
        throw new Error(`Replacement not found "${i}"`);
    }
    const importStr = typeof i === 'string' ? i : i[0];
    const IMP_MATCH = `/CallExpression[//Identifier[@name=='import' || @name=='importNotebook']]
        //Literal[@value == '${importStr}']`;
    debugger;
    var impParent = getArrayAST(`//*[${IMP_MATCH}]`, ast)[0];
    var impCall = getArrayAST(IMP_MATCH, impParent)[0];
    
    if(result.length > 1 || (i.includes('.ipynb') && !i.includes('['))) {
        const imports = result.map(r => `require('./${r.name = niceName(r)}')`).join(', ');
        impParent.splice(impParent.indexOf(impCall),
                         1,
                         getArrayAST('/*', multipleImportTemplate(imports)))
    } else {
        result = result[0] || result;
        impParent.splice(impParent.indexOf(impCall),
                         1,
                         getArrayAST('/*', `require('./${result.name = niceName(result)}')`));
    }
}

// TODO: use an AST for all of this static analysis
function replaceImports(exports, cell, allImports) {
    cell.name = niceName(cell);
    // TODO: use AST for this, doesn't work in quotes
    if(cell.filename.includes('get all cached')) {
        return cell.code;
    }
    var ast = getArrayAST('/*', cell.code);
    replaceCore(ast);
    exports.forEach(i => replaceImport(ast, i, allImports));
    addImports(ast, allImports);
    return escodegen.generate(ast);
}
module.exports = replaceImports;


export deploy notebook coordinator?

TODO: automatically find missing imports from distrib/Github/3rd party sources in any language?



In [ ]:
// assuming you've already run `aws configure`
var importer = require('../Core');
var {
    getImportsRecursively,
    replaceImports,
    injectImports,
    projectRelatives,
    outputExport
} = importer.import([
    'get imports recursively',
    'replace notebook imports',
    'inject cells notebooks webpack',
    'fix project paths',
    'output exported code'
]);

function exportAndDeploy(search, projectOutput, matchOutput) {
    // these are the top level cells starting the import tree
    const entryCells = importer.interpret(search);
    const interpretedImports = {};
    const imports = getImportsRecursively(interpretedImports, entryCells || [], search);
    
    // only include core cache files if interpret notebook is used, like in RPC handlers
    if(Object.keys(interpretedImports).filter(k => k.includes('interpret questions')).length > 0) {
        Object.assign(interpretedImports, injectImports(imports, interpretedImports));
    }

    var core = [];
    Object.keys(imports)
        .filter(i => interpretedImports[i].questions.length > 0
               || interpretedImports[i].questions[0].trim() != '')
        .forEach((e, i) => {
            const exportedCode = replaceImports(imports[e], interpretedImports[e], interpretedImports);
            outputExport(exportedCode, interpretedImports[e], projectOutput, matchOutput || {})
        });
    
    // TODO: output packed cells and cache
    // TODO: zip and upload to AWS
    return projectRelatives(projectOutput);
    //return delint(PROJECT_PATH)
}

module.exports = exportAndDeploy;


authoring header template?


In [ ]:

// TODO: move this heading authoring template to utility function
function authorTemplate(markdown, exportedCode) {
    return `
/**
 * Written by Brian Cullinan, exported using magic.
 * Copyright (c) ${(new Date()).getFullYear()} by Brian Cullinan, All rights reserved.
 *
${((markdown || '') + '').split('\n').map(l => ' * ' + l).join('\n')}
 *
 **/

` + exportedCode;
}

module.exports = authorTemplate;

output exported code?

In [ ]:
var fs = require('fs');
var os = require('os');
var path = require('path');
var minimatch = require('minimatch');
var importer = require('../Core')
var mkdirpSync = importer.import('mkdirp');
var authorTemplate = importer.import('authoring header template');

var getDirectory = (match) => match[match.length-1] === '/'
    ? match
    : (fs.existsSync(match) && fs.lstatSync(match).isDirectory()
       ? match
       : path.dirname(match));


// TODO: move this to filesystem utility?
var matchFilename = (filename, matchOutput, projectOutput) => Object.keys(matchOutput)
    .filter(match => minimatch(filename, match))
    .map(match => {
        var projectMatch = path.join(projectOutput, matchOutput[match])
        var dir = getDirectory(projectMatch);
        return path.join(dir, path.basename(projectMatch) === path.basename(dir)
            ? filename
            : path.basename(matchOutput[match]));
    });

function outputExport(exportedCode, cell, projectOutput, matchOutput) {
    // some special exceptions with file-naming
    if(cell.name.match(/^\./)) {
        throw new Error('No filename! ' + JSON.stringify(cell));
    }
    if(cell.name.toLowerCase() === 'readme.js') {
        exportedCode = cell.markdown;
        cell.name = 'readme.md';
    }
    if(cell.name.toLowerCase() === 'package.js'
       || cell.name.toLowerCase() === 'package.json'
       || cell.name.toLowerCase() === 'package-json.js') {
        cell.name = 'package.json';
    }
    projectOutput = projectOutput || path.join(path.resolve(__dirname), '../.output');
    
    // emit the file in every location request
    matchFilename(cell.name, matchOutput, projectOutput).forEach(filename => {
        // create directory if needed
        mkdirpSync(path.dirname(filename));
        console.log(`emitting ${filename}`);
        // add a code block header
        if(path.extname(filename) == '.js' || path.extname(filename) == '.cs') {
            exportedCode = authorTemplate(cell.markdown, exportedCode)
        }
        fs.writeFileSync(filename, exportedCode);
    })
}

module.exports = outputExport;


inject cells in to notebooks when built with webpack?


In [ ]:
var importer = require('../Core');
var getPermissions = importer.import('rpc permissions')
var path = require('path');

function camelCase(description) {
    return description.split(/\s+/ig).map((d, i) => i > 0 
                                          ? (d[0].toUpperCase() + d.substr(1)) 
                                          : d).join('')
}

// basic requirements for injecting different code into importer
var counter = 0;
function templateInject(description, code) {
    result = {};
    var name = camelCase(description);
    result[`${description}.ipynb[${counter}]`] = {
        markdown: `How to ${description}?`.split(/\n/ig).map(c => c + '\n'),
        language: 'javascript',
        cell_type: 'code',
        questions: [description],
        exports: [name],
        code: `
function ${name}() {
    return ${code};
}
module.exports = ${name};
`,
        filename: `../Utilities/notebook.ipynb/${description}[${counter}]`
    }
    counter++;
    return result;
}

// inject cellCache and cellIds loaded in to the interpreter so notebooks don't need to be uploaded, and webpack can tree-shake on functions already packed
// TODO: this should be a test in the Core notebook
function injectImports(imports, interpretedImports) {    
    const requires = [];
    const cells = Object.keys(interpretedImports).reduce((acc, k) => {
        const book = typeof interpretedImports[k].length === 'undefined'
            ? [interpretedImports[k]]
            : interpretedImports[k];
        if(typeof acc[book[0].filename] === 'undefined') {
            acc[book[0].filename] = [];
            const notebook = importer.interpret(path.basename(book[0].filename));
            notebook.forEach(cell => {
                // reference the function directly because it will be replaced by a file reference
                if(typeof imports[cell.id] !== 'undefined') {
                    requires.push(JSON.stringify(cell.id) + ': (ctx) => importer.import(' + JSON.stringify(cell.id) + ')');
                }
                cell.markdown.forEach(m => {
                    acc[cell.filename].push({
                        source: m.split(/\n/ig).map(l => l + '\n'),
                        language: cell.language,
                        cell_type: 'markdown'
                    });
                });
                acc[cell.filename].push({
                    source: cell.code.split(/\n/ig).map(l => l + '\n'),
                    language: cell.language,
                    cell_type: 'code'
                });
            });
        }
        return acc;
    }, {});
    
    const all = getPermissions();
    const permissions = Object.keys(all)
        .filter(k => typeof imports[all[k][1]] !== 'undefined')
        .reduce((acc, k) => (acc[k] = all[k], acc), {});

    imports['get injected.ipynb[0]'] = Object.keys(imports)
        .filter(k => !k.includes('get injected'));
    imports['get all cached.ipynb[1]'] = [];
    imports['get cached permissions.ipynb[2]'] = [];
    
    var injections = Object.assign({},
        templateInject('get injected', `{${requires.join(',\n        ')}}`),
        templateInject('get all cached', `${JSON.stringify(cells, null, 4)}`),
        templateInject('get cached permissions', `${JSON.stringify(permissions)}`));
    return injections;
}
module.exports = injectImports;
